In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Input

In [35]:
# import dataset and shuffle data
df = pd.read_csv('dataset.csv')
df = df.sample(frac=1).reset_index(drop=True)
df


,1 x 1,1 x 2,1 x 3,1 x 4,1 x 5,1 x 6,1 x 7,1 x 8,1 x 9,1 x 10,...,64 x 25,64 x 26,64 x 27,64 x 28,64 x 29,64 x 30,64 x 31,64 x 32,Subject,Position
0,0.446240,13.692313,42.882595,82.909018,119.565906,143.090204,155.796413,164.856172,174.180843,183.960268,...,117.403696,113.852474,112.555639,108.276288,94.759483,68.897652,37.960017,17.025853,5,0.0
1,0.736495,14.330287,42.273331,76.966306,105.643467,124.060810,137.007968,148.715817,159.634716,168.614379,...,132.740354,123.571813,120.406174,116.309863,103.363950,76.477008,42.833982,19.605986,5,0.0
2,0.052173,0.148675,0.391512,0.720109,0.889865,0.776884,0.660128,0.981694,1.957169,3.421697,...,9.876006,6.758157,4.339806,3.066950,2.430356,1.752784,1.093093,0.729911,5,0.0
3,0.104305,0.123357,0.192476,0.379924,0.653257,0.784854,0.621273,0.327119,0.140987,0.114232,...,0.289646,0.230582,0.173154,0.159602,0.159117,0.121386,0.062227,0.024475,9,0.0
4,0.078743,0.165612,0.315775,0.479040,0.605528,0.692313,0.801077,0.995093,1.260358,1.558109,...,4.495714,1.780931,1.125256,1.013136,0.815444,0.487389,0.208581,0.075842,4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2670,0.000000,0.000007,0.000012,0.000017,0.000065,0.000327,0.001222,0.003497,0.008722,0.020292,...,406.750162,500.000000,424.871385,251.503352,105.865583,33.219644,8.770848,2.843630,9,2.0
2671,0.000076,0.000318,0.000965,0.001890,0.002469,0.002765,0.005260,0.015007,0.037451,0.075832,...,1.927529,1.775748,1.456655,1.076371,0.736319,0.426972,0.186044,0.066032,1,0.0
2672,0.434854,0.430446,0.395607,0.355922,0.368936,0.462172,0.617454,0.803142,1.045489,1.374816,...,9.229834,5.362720,2.618059,1.444085,1.028548,0.701442,0.414271,0.274302,11,2.0
2673,3.856951,22.987691,62.445821,110.572879,146.859913,164.472888,172.797020,179.762925,185.316079,187.356175,...,117.538088,117.425527,116.081749,110.836712,96.649727,70.367944,38.926610,17.568193,1,0.0


In [36]:
# Split dataset in train and val
num_col = len(df.columns)
train_split = 2000
test_split = 2500

train_data = df.iloc[0:train_split, 0:num_col-2]
val_data = df.iloc[train_split:test_split, 0:num_col-2]

train_labels_sub = df.iloc[0:train_split, num_col-2:num_col-1]
train_labels_pos = df.iloc[0:train_split, num_col-1:num_col]
val_labels_sub = df.iloc[train_split:test_split, num_col-2:num_col-1]
val_labels_pos = df.iloc[train_split:test_split, num_col-1:num_col]


test_data = df.iloc[test_split:,:num_col-2]
result = df.iloc[test_split:, num_col-3:num_col]


In [37]:
tf_train = tf.data.Dataset.from_tensor_slices((train_data.values, train_labels_sub.values, train_labels_pos.values))
tf_val = tf.data.Dataset.from_tensor_slices((val_data.values, val_labels_sub.values, val_labels_pos.values))

print(tf_train)
print(tf_val)


<TensorSliceDataset shapes: ((2048,), (1,), (1,)), types: (tf.float64, tf.int64, tf.float64)>
<TensorSliceDataset shapes: ((2048,), (1,), (1,)), types: (tf.float64, tf.int64, tf.float64)>


In [5]:
plt.figure(figsize=(10,5))
i = 0

for pressure_map, label in tf_train.take(5):
    plt.subplot(1,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)

    plt.imshow(pressure_map.numpy().reshape((64, 32)), cmap='gray')
    plt.xlabel(label.numpy())
    
    i = i + 1


ValueError: too many values to unpack (expected 2)

<Figure size 720x360 with 0 Axes>

In [6]:
def preprocess_map(pressure_map, sub, pos):
    pressure_map = tf.reshape(pressure_map, [64, 32, 1])
    pressure_map = tf.cast(pressure_map, tf.float32) / 500.
    
    return pressure_map, sub, pos

tf_train = tf_train.map(preprocess_map, num_parallel_calls=tf.data.experimental.AUTOTUNE)
tf_val = tf_val.map(preprocess_map, num_parallel_calls=tf.data.experimental.AUTOTUNE)

print(tf_train)
print(tf_val)


<ParallelMapDataset shapes: ((64, 32, 1), (1,), (1,)), types: (tf.float32, tf.int64, tf.float64)>
<ParallelMapDataset shapes: ((64, 32, 1), (1,), (1,)), types: (tf.float32, tf.int64, tf.float64)>


In [7]:
def pipeline(tf_data):
    tf_data = tf_data.shuffle(100)
    tf_data = tf_data.batch(1)
    tf_data = tf_data.prefetch(tf.data.experimental.AUTOTUNE)
    
    return tf_data

tf_train = pipeline(tf_train)
tf_val = pipeline(tf_val)

print(tf_train)
print(tf_val)


<PrefetchDataset shapes: ((None, 64, 32, 1), (None, 1), (None, 1)), types: (tf.float32, tf.int64, tf.float64)>
<PrefetchDataset shapes: ((None, 64, 32, 1), (None, 1), (None, 1)), types: (tf.float32, tf.int64, tf.float64)>


In [8]:
'''model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(64, 32, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.4),
    
    Conv2D(16, (3, 3), activation='relu', padding='valid'),
    MaxPooling2D((2, 2)),
    Dropout(0.4),
    
    Flatten(),
    Dense(120, activation='relu'),
    Dense(84, activation='relu'),
    
    Dropout(0.4),
    Dense(14, activation='softmax'),
    #Dense(4, activation='softmax'),
])'''

"model = Sequential([\n    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(64, 32, 1)),\n    MaxPooling2D((2, 2)),\n    Dropout(0.4),\n    \n    Conv2D(16, (3, 3), activation='relu', padding='valid'),\n    MaxPooling2D((2, 2)),\n    Dropout(0.4),\n    \n    Flatten(),\n    Dense(120, activation='relu'),\n    Dense(84, activation='relu'),\n    \n    Dropout(0.4),\n    Dense(14, activation='softmax'),\n    #Dense(4, activation='softmax'),\n])"

In [19]:
inp = Input((64, 32, 1),)

x = Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(64, 32, 1))(inp)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.4)(x)
    
x = Conv2D(16, (3, 3), activation='relu', padding='valid')(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.4)(x)

x = Flatten()(x)
x = Dense(120, activation='relu')(x)
x = Dense(84, activation='relu')(x)

x = Dropout(0.4)(x)

out = Dense(2, activation='sigmoid')(x)

out1 = Dense(14, activation='softmax')(out)
out2 = Dense(4, activation='softmax')(out)

model = Model(inp, [out1, out2])

In [20]:
optimiser = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimiser, loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 64, 32, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 64, 32, 32)   320         input_4[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 32, 16, 32)   0           conv2d_6[0][0]                   
__________________________________________________________________________________________________
dropout_9 (Dropout)             (None, 32, 16, 32)   0           max_pooling2d_6[0][0]            
____________________________________________________________________________________________

In [29]:
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=2, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, verbose=1),
]


Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [22]:
train_ = model.fit(tf_train, validation_data=tf_val, epochs=15, callbacks = callbacks)


ValueError: in converted code:

    /usr/local/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_v2.py:677 map_fn
        batch_size=None)
    /usr/local/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training.py:2469 _standardize_tensors
        exception_prefix='target')
    /usr/local/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_utils.py:539 standardize_input_data
        str(data)[:200] + '...')

    ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 2 array(s), for inputs ['dense_15', 'dense_16'] but instead got the following list of 1 arrays: [<tf.Tensor 'args_1:0' shape=(None, 1) dtype=int64>]...


In [ ]:
plt.plot(train_log.history['accuracy'], label='accuracy')
plt.plot(train_log.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

print('Training accuracy: %f' % train_log.history['accuracy'][-1])
print('Validation accuracy: %f' % train_log.history['val_accuracy'][-1])

In [ ]:
tf_test_data = tf.data.Dataset.from_tensor_slices(([test_data.to_numpy().reshape(len(test_data), 64, 32, 1)]))

predictions = model.predict(tf_test_data)
predictions = np.argmax(predictions, axis=1)


In [ ]:
plt.figure(figsize=(10,5))

for i, row in test_data.head(20).reset_index(drop=True).iterrows():
    plt.subplot(4,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)

    plt.imshow(row.values.reshape((64, 32)), cmap='gray')
    plt.xlabel(predictions[i])

test_data.head(10).reset_index(drop=True)


In [ ]:
result.head(20).reset_index(drop=True)